## 1. IMPORTACIÓN DE LIBRERÍAS ESENCIALES



**Cuándo usar:** Al inicio de cualquier notebook de análisis de datos.



**Propósito:** Cargar todas las librerías necesarias para manipulación de datos, visualización, estadística y machine learning básico.


In [ ]:
# Librerías fundamentales
import numpy as np                    # Operaciones numéricas y arrays
import pandas as pd                   # Manipulación de dataframes
import re                             # Expresiones regulares (limpieza de texto)

# Visualización
import matplotlib.pyplot as plt       # Gráficos básicos
import seaborn as sns                 # Gráficos estadísticos avanzados

# Estadística
import scipy.stats                    # Tests estadísticos


# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Configuración
import warnings
warnings.filterwarnings('ignore')     # Ocultar warnings molestos

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")

## 2. CARGA DE DATOS



**Cuándo usar:** Para leer archivos CSV u otros formatos de datos.



**Propósito:** Importar datasets desde archivos, especificando separadores y rutas correctas.


In [ ]:
# Lectura de CSV con separador personalizado
df = pd.read_csv('ruta/al/archivo.csv', sep=',')

# Alternativas comunes:
df = pd.read_csv('archivo.csv', sep=';')      # Separador punto y coma
df = pd.read_csv('archivo.csv', sep='\	')     # Separador tabulación
df = pd.read_excel('archivo.xlsx')            # Para Excel

# Inspección inicial
print(f"Dimensiones: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")
df.head()  # Primeras 5 filas


## 3. LIMPIEZA DE DATOS CON EXPRESIONES REGULARES



**Cuándo usar:** Cuando hay que limpiar columnas con texto que contienen caracteres especiales, símbolos monetarios, unidades, etc.



**Propósito:** Convertir strings con formato inconsistente a valores numéricos utilizables.


In [ ]:
# PATRÓN: Eliminar caracteres no deseados y convertir a numérico

# Ejemplo: Columna con formato "$123.45M" -> 123.45
def limpiar_columna_numerica(valor):
    """
    Limpia strings con símbolos y los convierte a float
    Ej: "$45.2M" -> 45.2
    """

    # Eliminar símbolo $ y luego M (o cualquier letra)
    limpio = re.sub("\$", "", str(valor))  # Quitar $
    limpio = re.sub("[A-Za-z]", "", limpio) # Quitar letras (M, K, etc.)
    return float(limpio)



# Aplicar a toda una columna (opción 1: con apply)
df['columna_limpia'] = df['columna_sucia'].apply(limpiar_columna_numerica)

# Aplicar a toda una columna (opción 2: con list comprehension - más rápido)
df['columna_limpia'] = [float(re.sub("M", "", re.sub("\$", "", str(x)))) for x in df['columna_sucia'].values]   

# IMPORTANTE: Filtrar valores problemáticos ANTES de convertir
# Eliminar filas con valores como "Unknown", "N/A", etc.
df = df[df['columna'] != "Unknown"].reset_index(drop=True)
df = df[df['columna'] != "N/A"].reset_index(drop=True)

## 4. CREACIÓN DE VARIABLES DERIVADAS (FEATURE ENGINEERING)



**Cuándo usar:** Para crear nuevas columnas basadas en transformaciones de las existentes.



**Propósito:** Generar variables más útiles para el análisis (ej: décadas desde años, categorías desde rangos, etc.).


In [ ]:
# PATRÓN 1: Agrupar años en décadas
# Si tienes años: 1987, 1993, 2001 -> décadas: 1980, 1990, 2000
df['decada'] = df['año'] - df['año'] % 10

# PATRÓN 2: Crear rangos/bins
df['rango_edad'] = pd.cut(df['edad'], 
                          bins=[0, 18, 35, 50, 100], 
                          labels=['Niño', 'Joven', 'Adulto', 'Mayor'])

# PATRÓN 3: Categorizar basado en condiciones
df['categoria'] = df['valor'].apply(lambda x: 'Alto' if x > 50 else 'Bajo')

# PATRÓN 4: Extraer información de fechas
df['fecha'] = pd.to_datetime(df['fecha'])
df['año'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['dia_semana'] = df['fecha'].dt.dayofweek

## 5. AGREGACIONES Y GROUPBY



**Cuándo usar:** Para calcular estadísticas por grupos (conteos, promedios, sumas).



**Propósito:** Resumir datos agrupados por una o más categorías.


In [ ]:
# PATRÓN 1: Contar elementos por grupo
conteos = df.groupby(['categoria1', 'categoria2']).count()['columna_a_contar'].reset_index()
conteos.columns = ['categoria1', 'categoria2', 'conteo']  # Renombrar para claridad

# PATRÓN 2: Calcular promedios por grupo
promedios = df.groupby('categoria')['variable_numerica'].mean().reset_index()

# PATRÓN 3: Múltiples agregaciones a la vez
resumen = df.groupby('categoria').agg({
    'columna1': 'mean',
    'columna2': 'sum',
    'columna3': 'count',
    'columna4': ['min', 'max']
}).reset_index()

# PATRÓN 4: Filtrar grupos según condición
# Ejemplo: Mantener solo categorías con más de N apariciones
conteo_por_categoria = df.groupby('categoria')['id'].count()
categorias_frecuentes = conteo_por_categoria[conteo_por_categoria > 100].index
df_filtrado = df[df['categoria'].isin(categorias_frecuentes)]

## 6. SELECCIÓN DE TOP N POR GRUPO



**Cuándo usar:** Cuando necesitas los N elementos más altos/bajos de cada categoría.



**Propósito:** Extraer rankings dentro de grupos (ej: top 5 productos por categoría).


In [ ]:
# PATRÓN: Obtener top N elementos por cada grupo
resultado_final = pd.DataFrame()  # DataFrame vacío para acumular

# Iterar sobre cada grupo único
for grupo in df['categoria'].unique():
    # Filtrar datos del grupo actual
    datos_grupo = df[df['categoria'] == grupo].copy()

    # Ordenar por la métrica deseada y tomar top N
    top_n = min(5, len(datos_grupo))  # Protección si hay menos de 5 elementos
    top_elementos = datos_grupo.sort_values('metrica', ascending=False).head(top_n)

    # Añadir al resultado acumulado
    resultado_final = pd.concat([resultado_final, top_elementos], ignore_index=True)

# Alternativa más eficiente con groupby (sin loop)
top_por_grupo = (df.sort_values('metrica', ascending=False)
                   .groupby('categoria')
                   .head(5)  # Top 5 de cada grupo
                   .reset_index(drop=True))

## 7. VISUALIZACIÓN: GRÁFICOS DE BARRAS CON FACETAS



**Cuándo usar:** Para comparar categorías a través de múltiples grupos/subconjuntos.



**Propósito:** Crear múltiples subgráficos (facetas) organizados por una variable categórica.


In [ ]:
# BARPLOT CON FACETAS (catplot)
# Útil para mostrar distribuciones de categorías divididas por otra variable

sns.catplot(
    kind="bar",              # Tipo: barplot (también: box, violin, strip, etc.)
    data=df,                 # DataFrame con los datos
    x="categoria",           # Variable en eje X (categórica)
    y="valor",               # Variable en eje Y (numérica)
    col="grupo",             # Variable para crear facetas (columnas)
    col_wrap=4,              # Número de columnas antes de saltar a nueva fila
    hue="categoria",         # Color según categoría (opcional)
    sharex=False,            # Ejes X independientes por faceta
    sharey=False,            # Ejes Y independientes por faceta
    legend=False,            # Ocultar leyenda si es redundante
    height=4,                # Altura de cada faceta
    aspect=1.2               # Relación ancho/alto
)

plt.tight_layout()
plt.show()

## 8. VISUALIZACIÓN: LÍNEAS TEMPORALES CON GRUPOS



**Cuándo usar:** Para mostrar evolución temporal de múltiples grupos/categorías.



**Propósito:** Visualizar tendencias a lo largo del tiempo, diferenciando por colores.


In [ ]:
# LINEPLOT CON MÚLTIPLES SERIES
# Perfecto para series temporales comparando grupos

# IMPORTANTE: Filtrar categorías poco frecuentes antes de graficar
# Esto evita gráficos saturados y mejora la interpretabilidad
conteo_categorias = df.groupby('categoria')['id'].count()
categorias_relevantes = conteo_categorias[conteo_categorias > 200].index
df_filtrado = df[df['categoria'].isin(categorias_relevantes)]

# Crear el gráfico de líneas
sns.relplot(
    kind="line",             # Tipo: line (también: scatter)
    data=df_filtrado,        # Datos filtrados
    x="tiempo",              # Variable temporal (eje X)
    y="valor",               # Variable numérica (eje Y)
    hue="categoria",         # Colorear por categoría
    style="categoria",       # Estilo de línea por categoría (opcional)
    markers=True,            # Mostrar marcadores en puntos (opcional)
    dashes=False,            # Líneas sólidas (True para discontinuas)
    height=5,
    aspect=1.5
)

plt.title("Evolución temporal por categoría")
plt.show()

# NOTA: relplot con kind="line" calcula automáticamente la MEDIA 
# y el intervalo de confianza para cada punto temporal

## 9. TESTS ESTADÍSTICOS: NORMALIDAD Y COMPARACIÓN DE GRUPOS



**Cuándo usar:** Para determinar si hay diferencias significativas entre grupos.



**Propósito:** Aplicar el test estadístico apropiado según la distribución de los datos.


In [ ]:
# PASO 1: COMPROBAR NORMALIDAD
# Importante: Los tests paramétricos (t-test, ANOVA) requieren normalidad

# Visualización de distribución
df['variable'].hist(bins=50, edgecolor='black')
plt.title("Distribución de la variable")
plt.show()

# Test de Shapiro-Wilk (H0: los datos son normales)
statistic, p_value = scipy.stats.shapiro(df['variable'])
print(f"Shapiro-Wilk p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Los datos NO son normales -> Usar tests NO paramétricos")
else:
    print("Los datos SÍ son normales -> Usar tests paramétricos")

# REGLA GENERAL:
# p-value < 0.05 -> Rechazamos H0 -> NO normalidad -> Tests no paramétricos
# p-value >= 0.05 -> No rechazamos H0 -> Normalidad -> Tests paramétricos

## 10. TESTS ESTADÍSTICOS: UN GROPO VS RESTO

In [ ]:
# PASO 2: COMPARAR UN GRUPO VS EL RESTO (1 vs All)
# Útil para detectar si una categoría es significativamente diferente

from scipy.stats import ranksums, ttest_ind

# Para datos NO NORMALES: Wilcoxon rank-sum test (Mann-Whitney)
categorias = df['categoria'].unique()
for cat in categorias:
    grupo_actual = df['variable'][df['categoria'] == cat]
    resto = df['variable'][df['categoria'] != cat]

    # Test de rangos (NO paramétrico)
    _, p_val = ranksums(grupo_actual, resto, alternative='greater')

    if p_val < 0.05:
        print(f"{cat} es significativamente MAYOR que el resto (p={p_val:.4f})")



# Para datos NORMALES: T-test
_, p_val = ttest_ind(grupo_actual, resto, alternative='greater')


# Alternativas para 'alternative':
# - 'greater': grupo_actual > resto
# - 'less': grupo_actual < resto  
# - 'two-sided': grupo_actual ≠ resto (diferente, sin dirección)

In [ ]:
# PASO 3: COMPARAR MÚLTIPLES GRUPOS SIMULTÁNEAMENTE
from scipy.stats import kruskal, f_oneway

# Preparar grupos como listas separadas
grupos = [df['variable'][df['categoria'] == cat].values 
          for cat in df['categoria'].unique()]


# Para datos NO NORMALES: Kruskal-Wallis (ANOVA no paramétrico)
statistic, p_val = kruskal(*grupos)
print(f"Kruskal-Wallis p-value: {p_val:.4f}")


# Para datos NORMALES: ANOVA
statistic, p_val = f_oneway(*grupos)


if p_val < 0.05:
    print("Al menos un grupo es significativamente diferente")
else:
    print("No hay diferencias significativas entre grupos")


# RESUMEN DE TESTS:
# ┌─────────────────┬──────────────────┬──────────────────────┐
# │  Comparación    │  Paramétrico     │  No Paramétrico      │
# ├─────────────────┼──────────────────┼──────────────────────┤
# │  2 grupos       │  t-test          │  ranksums (Wilcoxon) │
# │  3+ grupos      │  ANOVA           │  Kruskal-Wallis      │
# └─────────────────┴──────────────────┴──────────────────────┘

## 11. PCA (ANÁLISIS DE COMPONENTES PRINCIPALES)



**Cuándo usar:** Para reducir dimensionalidad de datos con muchas variables, especialmente tras one-hot encoding.



**Propósito:** Proyectar datos de alta dimensión a 2D/3D para visualización y detectar patrones/clusters.


In [ ]:
# PCA: REDUCCIÓN DE DIMENSIONALIDAD
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# PASO 1: Estandarizar los datos (CRÍTICO para PCA)
# PCA es sensible a la escala, por lo que todas las variables deben estar
# en la misma escala (media=0, varianza=1)
scaler = StandardScaler()
X_estandarizado = scaler.fit_transform(matriz_de_features)

# PASO 2: Aplicar PCA
pca = PCA(n_components=2)  # Reducir a 2 dimensiones para visualización
X_pca = pca.fit_transform(X_estandarizado)


# Información sobre la varianza explicada
print(f"Varianza explicada por PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"Varianza explicada por PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"Varianza total explicada: {pca.explained_variance_ratio_.sum():.2%}")

# PASO 3: Crear DataFrame con componentes principales + variables originales
df_pca = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'variable_color': df['categoria'].values,       # Para colorear gráfico
    'variable_continua': df['valor_numerico'].values  # Para gradiente de color
})

# PASO 4: Visualizar resultados
# Gráfico 1: Coloreado por variable categórica
sns.relplot(data=df_pca, x='PC1', y='PC2', hue='variable_color', 
            height=6, aspect=1.3)
plt.title("PCA coloreado por categoría")
plt.show()

# Gráfico 2: Coloreado por variable continua (gradiente)
sns.relplot(data=df_pca, x='PC1', y='PC2', hue='variable_continua',
            palette='viridis', height=6, aspect=1.3)
plt.title("PCA coloreado por variable numérica")
plt.show()

# INTERPRETACIÓN:
# - Si hay clusters visibles -> los datos tienen estructura/agrupaciones
# - Si el color (categoría) coincide con clusters -> las categorías son distinguibles
# - PC1 y PC2 son las direcciones de máxima varianza en los datos

## 12. FLUJO COMPLETO: MATRIZ ONE-HOT + FILTRADO + PCA



**Cuándo usar:** Pipeline completo para analizar datos categóricos de alta dimensión.



**Propósito:** Desde texto con múltiples valores hasta visualización en 2D pasando por filtrado y reducción dimensional.


In [ ]:
# PIPELINE COMPLETO: ONE-HOT -> FILTRADO -> PCA -> VISUALIZACIÓN

# 1) One-hot encoding de columna con múltiples valores
matriz_oh = df['columna_multivalor'].str.get_dummies(sep=", ")

# 2) Filtrar valores poco frecuentes (aparecer en > N filas)
matriz_oh_filtrada = matriz_oh.loc[:, matriz_oh.sum(axis=0) > 10]

# 3) Estandarizar
X_std = StandardScaler().fit_transform(matriz_oh_filtrada)

## 13. TIPS Y BUENAS PRÁCTICAS 

### Manipulación de datos:

- **Siempre** usar `.copy()` cuando trabajes con subconjuntos para evitar SettingWithCopyWarning

- Preferir list comprehension sobre `.apply()` para mejor rendimiento en operaciones simples

- Usar `.isin()` para filtrar por múltiples valores es más eficiente que múltiples condiciones OR

- Filtrar columnas por % de NAs antes de eliminar filas ahorra datos

- Usar `.sample()` frecuentemente durante desarrollo para verificar transformaciones

- Desarrollar funciones complejas paso a paso, probando con 1-2 casos antes de apply



### Fechas y tiempo:

- Siempre convertir fechas a datetime con `pd.to_datetime()`

- Para UNIX timestamps usar `unit='s'` (segundos) o `unit='ms'` (milisegundos)

- Usar `.dt` para extraer componentes (year, month, day, hour, etc.)

- Verificar completitud de datos temporales (último año puede estar incompleto)

- Ordenar por fecha con `sort_index()` antes de graficar series temporales



### Tests estadísticos:

- **Siempre** verificar normalidad antes de elegir test (Shapiro-Wilk)

- Si p < 0.05 en Shapiro → datos NO normales → usar tests NO paramétricos

- Ranksums/Wilcoxon para 2 grupos (no paramétrico)

- Kruskal-Wallis para 3+ grupos (no paramétrico)

- T-test para 2 grupos (paramétrico, requiere normalidad)

- ANOVA para 3+ grupos (paramétrico, requiere normalidad)



### PCA:

- **SIEMPRE** estandarizar datos antes de PCA (StandardScaler)

- 2 componentes son suficientes para visualización

- Verificar varianza explicada para evaluar calidad de reducción

- PCA es útil para detectar outliers visualmente



### Machine Learning - Clasificación:

- Usar `class_weight='balanced'` en modelos para compensar desbalanceo

- OneHotEncoder de sklearn genera matrices sparse (eficientes en memoria)

- Validación cruzada (cv=5) ayuda a obtener estimaciones más robustas

- Para clasificación: LogisticRegressionCV selecciona automáticamente el mejor parámetro C



### Machine Learning - Regresión:

- Estandarizar Y es opcional pero ayuda a convergencia

- ElasticNet combina L1 (feature selection) y L2 (regularización)

- l1_ratio=0 → Ridge puro, l1_ratio=1 → Lasso puro

- ElasticNetCV encuentra automáticamente alpha y l1_ratio óptimos

- Visualizar siempre real vs predicho para detectar patrones en errores

- Distribución de errores centrada en 0 = modelo no sesgado

- Comparar con regresión lineal simple para evaluar beneficio de regularización

- Analizar gap train-test para detectar overfitting



### Clustering:

- **SIEMPRE** estandarizar datos antes de K-means (sensible a escalas)

- Usar Silhouette Score para seleccionar K óptimo

- Considerar complejidad: a veces K menor es más interpretable

- Excluir variables identificadoras o targets del clustering

- Visualizar con PCA para evaluar calidad visual de separación

- `n_init='auto'` en sklearn reciente (antes era n_init=10)

- K-means puede encontrar óptimos locales, usar random_state para reproducibilidad



### Detección de outliers:

- Identificar outliers por errores grandes en predicciones

- Analizar qué features contribuyen a la anomalía

- Evaluar si eliminar outliers mejora significativamente el modelo

- No eliminar outliers sin justificación (pueden ser datos válidos)



### Agregaciones y análisis de grupos:

- `pd.NamedAgg` hace el código más legible y explícito

- Usar funciones lambda para cálculos personalizados: `lambda x: (x == True).mean() * 100`

- `value_counts().nlargest(N)` es más eficiente que `value_counts().sort_values().head(N)`

- `.agg()` permite aplicar múltiples funciones a la vez

- Usar `sort_index()` vs `sort_values()` apropiadamente:

  - `sort_index()` → ordenar por categorías/índices (ej: años cronológicamente)

  - `sort_values()` → ordenar por valores/frecuencias

- **CRÍTICO**: Filtrar categorías con pocas muestras antes de agregar (ej: >= 5 apariciones)

- Verificar tamaño de muestra con `describe()` para variables categóricas

- Cuidado con sesgos: normalizar métricas cuando sea necesario (ej: comentarios por vista)



### Documentación y debugging:

- **SIEMPRE** leer documentación de funciones nuevas

- Usar `help(función)` para ver documentación en Python

- Probar código en piezas pequeñas antes de aplicar a todo el dataset

- Usar `.sample()` para verificar transformaciones durante desarrollo

- Verificar tipos de datos con `type()` y `.dtypes`

- Comprobar que resultados tienen sentido (estadísticas descriptivas, extremos)

### Orden recomendado en análisis con ML:
1. Cargar datos y exploración inicial (shape, head, info, describe)
2. Limpieza (valores faltantes, conversión de tipos, regex)
3. Feature engineering (crear variables derivadas)
4. Análisis exploratorio (groupby, agregaciones, visualizaciones)
5. Tests estadísticos (normalidad, comparaciones)
6. Preparación para ML (one-hot encoding, train/test split)
7. Entrenamiento de modelos (con validación cruzada)
8. Evaluación (múltiples métricas: AUC-PR, AUC-ROC, confusion matrix)
9. Visualización de resultados (curvas, distribuciones)

In [ ]:
### Expresiones regulares comunes:
re.sub("\$", "", texto)         # Eliminar símbolo $
re.sub("[A-Za-z]", "", texto)   # Eliminar todas las letras

---

# PARTE II: MACHINE LEARNING Y MODELADO PREDICTIVO

---


## 14. LIMPIEZA DE DATOS: SELECCIÓN DE COLUMNAS Y ELIMINACIÓN DE NULOS



**Cuándo usar:** Al inicio del análisis para preparar un dataset limpio.



**Propósito:** Seleccionar solo las columnas relevantes y eliminar filas con valores faltantes.


In [ ]:
# PATRÓN: Seleccionar columnas específicas y eliminar nulos

# Definir columnas de interés
columnas_relevantes = ['col1', 'col2', 'col3', 'target']

# Seleccionar y limpiar en un solo paso
df_limpio = df[columnas_relevantes].dropna()

# Verificar resultado
print(f"Filas originales: {len(df)}")
print(f"Filas tras limpieza: {len(df_limpio)}")
print(f"Filas eliminadas: {len(df) - len(df_limpio)}")
print(f"Columnas seleccionadas: {df_limpio.columns.tolist()}")

## 15. SELECCIÓN DE TOP N VALORES MÁS FRECUENTES



**Cuándo usar:** Para enfocarse en las categorías más comunes y reducir ruido.



**Propósito:** Identificar y filtrar por los N valores con mayor frecuencia en una columna categórica.


In [ ]:
# PATRÓN: Obtener top N valores más frecuentes y filtrar dataset

# Paso 1: Contar y obtener top N (ej: top 5)
top_n_valores = df['columna_categorica'].value_counts().nlargest(5)
print("Top 5 categorías y sus frecuencias:")
print(top_n_valores)

# Paso 2: Extraer solo los índices (los valores/nombres)
top_n_nombres = top_n_valores.index
print(f"Nombres de top 5: {top_n_nombres.tolist()}")


# Paso 3: Filtrar el dataframe original
df_filtrado = df[df['columna_categorica'].isin(top_n_nombres)]

## 16. VISUALIZACIÓN: COUNTPLOT CON HUE



**Cuándo usar:** Para visualizar frecuencias de categorías divididas por otra variable.



**Propósito:** Mostrar conteos con barras apiladas o agrupadas coloreadas por subgrupos.


In [ ]:
# COUNTPLOT: Conteo de categorías con subdivisión por color
plt.figure(figsize=(10, 6))  # Tamaño de figura
sns.set_theme(style="darkgrid")  # Estilo de fondo

plot = sns.countplot(
    data=df,
    x='categoria_principal',      # Variable en eje X
    hue='subcategoria',           # Variable para colorear/dividir barras
    palette='Set2'                # Paleta de colores (opcional)
)

## 17. AGREGACIONES AVANZADAS CON pd.NamedAgg



**Cuándo usar:** Cuando necesitas aplicar múltiples funciones de agregación y nombrarlas claramente.



**Propósito:** Crear agregaciones complejas con nombres descriptivos de columnas, incluyendo funciones lambda personalizadas.


In [ ]:
# AGREGACIONES COMPLEJAS CON NamedAgg
# Útil para calcular múltiples métricas simultáneamente
resultado = df.groupby(['categoria1', 'categoria2']).agg(
    # Formato: nombre_columna_resultado=pd.NamedAgg(column='columna_origen', aggfunc=función)
    total_registros=pd.NamedAgg(
        column='cualquier_columna',
        aggfunc='count'
    ),
    porcentaje_verdaderos=pd.NamedAgg(
        column='columna_booleana',
        aggfunc=lambda x: (x == True).mean() * 100  # % de valores True
    ),
).reset_index()

print(resultado.head())

# FUNCIONES LAMBDA ÚTILES PARA BOOLEANOS:
# (x == True).mean() * 100   -> Porcentaje de True
# (x == False).mean() * 100  -> Porcentaje de False
# (x == valor).sum()         -> Conteo de un valor específico
# x.nunique()                -> Número de valores únicos

## 18. FILTRADO BASADO EN RESULTADO DE AGREGACIÓN



**Cuándo usar:** Cuando necesitas filtrar el dataframe original basándote en resultados de un groupby.



**Propósito:** Seleccionar subconjuntos de datos según criterios calculados en agregaciones.


In [ ]:
# PATRÓN: Filtrar dataset original según resultado de agregación
# Paso 1: Realizar agregación y obtener top N según criterio
agregado = df.groupby('categoria').agg(
    metrica=pd.NamedAgg(column='variable', aggfunc='mean')
).reset_index()

# Paso 2: Ordenar y seleccionar top N
top_categorias = (agregado
                  .sort_values('metrica', ascending=False)
                  .head(3)  # Top 3
                  ['categoria'])  # Solo la columna con nombres


# Paso 3: Filtrar dataframe original
df_filtrado = df[df['categoria'].isin(top_categorias)]

## 19. VISUALIZACIÓN: BARPLOT AGRUPADO CON HUE



**Cuándo usar:** Para mostrar valores agregados (no conteos) divididos por categorías y subgrupos.



**Propósito:** Visualizar métricas calculadas (promedios, porcentajes) con barras agrupadas.


In [ ]:
# BARPLOT: Para valores agregados (no conteos)
# Diferencia con countplot: barplot muestra valores de una columna numérica
plt.figure(figsize=(12, 6))
sns.set_theme(style="darkgrid")

plot = sns.barplot(
    data=df_agregado,            # DataFrame con datos ya agregados
    x='categoria_principal',     # Eje X: categorías principales
    y='metrica_calculada',       # Eje Y: valores numéricos (ej: porcentajes)
    hue='subcategoria',          # Dividir/colorear por subcategoría
    palette='viridis',           # Paleta de colores
    ci=None                      # Sin barras de error (datos ya agregados)
)

plt.tight_layout()
plt.legend(title='Subcategoría', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# NOTA: Si barplot recibe datos sin agregar, calculará la media automáticamente
# y mostrará barras de error (intervalo de confianza)

## 20. PREPARACIÓN DE DATOS PARA MACHINE LEARNING



**Cuándo usar:** Antes de entrenar cualquier modelo supervisado.



**Propósito:** Separar features (X) y target (y), y convertir tipos de datos apropiadamente.


In [ ]:
# PREPARACIÓN X e y PARA MODELOS
# Paso 1: Definir variable objetivo (y)
y = df['variable_objetivo']

# Paso 2: Definir features (X) eliminando la variable objetivo
X = df.drop(columns=['variable_objetivo'])

# Paso 3: Convertir y a tipo numérico si es necesario
# Para clasificación binaria (True/False -> 1/0)
y = y.astype(int)

# ALTERNATIVA: Convertir categorías a números con LabelEncoder
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y = le.fit_transform(y)

# Verificar shapes
print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"Tipo de y: {y.dtype}")
print("Distribución de clases:")
print(y.value_counts())
print(f"Columnas en X: {X.columns.tolist()}")

## 21. ONE-HOT ENCODING CON SCIKIT-LEARN



**Cuándo usar:** Para convertir variables categóricas en formato numérico para modelos de ML.



**Propósito:** Transformar categorías en matrices binarias (0/1) compatibles con algoritmos de ML.


In [ ]:
# ONE-HOT ENCODING CON SKLEARN (para ML)
# Diferencia con pandas: sklearn devuelve matriz sparse (más eficiente)
from sklearn.preprocessing import OneHotEncoder

# Datos categóricos (todas las columnas deben ser categóricas)
X_categorico = df[['columna1', 'columna2', 'columna3']]

# Paso 1: Crear y ajustar el encoder
encoder = OneHotEncoder(
    sparse_output=False,  
    handle_unknown='ignore'  # Ignorar categorías nuevas en test
)

# Paso 2: Fit y transform
X_encoded = encoder.fit_transform(X_categorico)

# Paso 3: Obtener nombres de las columnas generadas
feature_names = encoder.get_feature_names_out(input_features=X_categorico.columns)

## 22. DIVISIÓN TRAIN/TEST CON ESTRATIFICACIÓN



**Cuándo usar:** Antes de entrenar modelos, especialmente con datos desbalanceados.



**Propósito:** Dividir datos en conjuntos de entrenamiento y prueba manteniendo las proporciones de clases.


In [ ]:
# TRAIN/TEST SPLIT CON ESTRATIFICACIÓN
# CRÍTICO para datos desbalanceados
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,                    # Features
    y,                    # Target
    test_size=0.1,        # 10% para test, 90% para train
    stratify=y,           # IMPORTANTE: Mantener proporción de clases
    random_state=42       # Para reproducibilidad
)

# Otras opciones útiles:
# test_size=0.2          # 20% test (más común)
# test_size=0.3          # 30% test
# shuffle=True           # Mezclar antes de dividir (por defecto)
# random_state=None      # Sin seed (resultados no reproducibles)

## 23. REGRESIÓN LOGÍSTICA CON VALIDACIÓN CRUZADA Y DATOS DESBALANCEADOS



**Cuándo usar:** Para clasificación binaria con datos desbalanceados.



**Propósito:** Entrenar modelo de clasificación con regularización L2 (Ridge), validación cruzada y balance de clases.


In [ ]:
# REGRESIÓN LOGÍSTICA CON CV Y CLASS BALANCING
from sklearn.linear_model import LogisticRegressionCV

# Entrenar modelo
modelo = LogisticRegressionCV(
    Cs=10,                      # Número de valores de C (regularización) a probar
    cv=5,                       # 5-fold cross-validation
    random_state=42,            # Reproducibilidad
    solver='liblinear',         # Solver rápido para datasets pequeños/medianos
    max_iter=100,               # Iteraciones máximas
    penalty='l2',               # Regularización Ridge (L2)
    class_weight='balanced',    # CRÍTICO: Balancear clases automáticamente
    scoring='roc_auc'           # Métrica para seleccionar mejor C
).fit(X_train, y_train)

print(f"Mejor C encontrado: {modelo.C_[0]}")
print(f"Score en train: {modelo.score(X_train, y_train):.4f}") # Este es accuracy
print(f"Score en test: {modelo.score(X_test, y_test):.4f}")

# EXPLICACIÓN DE PARÁMETROS CLAVE:
# - Cs: Valores de regularización a probar (10 valores logarítmicamente espaciados)
# - cv: Número de folds para validación cruzada
# - penalty='l2': Regularización Ridge (penaliza coeficientes grandes)
# - penalty='l1': Regularización Lasso (selección de features)
# - class_weight='balanced': Ajusta pesos según frecuencia inversa de clases
#   peso_clase_i = n_samples / (n_classes * n_samples_clase_i)
# - solver opciones:
#   'liblinear': Bueno para datasets pequeños/medianos, soporta L1 y L2
#   'lbfgs': Default, bueno para multiclase
#   'saga': Bueno para datasets grandes

## 24. MÉTRICAS DE EVALUACIÓN: AUC-PR (PRECISION-RECALL)



**Cuándo usar:** Para evaluar modelos de clasificación, especialmente con datos desbalanceados.



**Propósito:** Calcular curvas de precisión-recall y su AUC, más informativa que ROC para datos desbalanceados.


In [ ]:
# CURVA PRECISION-RECALL Y AUC
# MÁS APROPIADA que ROC para datos desbalanceados
from sklearn.metrics import precision_recall_curve, auc

# OPCIÓN 1: Usando predicciones binarias (menos recomendado)
y_pred_train = modelo.predict(X_train)
precision_train, recall_train, _ = precision_recall_curve(y_train, y_pred_train)
auc_pr_train = auc(recall_train, precision_train)

y_pred_test = modelo.predict(X_test)
precision_test, recall_test, _ = precision_recall_curve(y_test, y_pred_test)
auc_pr_test = auc(recall_test, precision_test)

print(f"AUC-PR en train: {auc_pr_train:.4f}")
print(f"AUC-PR en test: {auc_pr_test:.4f}")

## 25. VISUALIZACIÓN: DENSIDAD DE PROBABILIDADES PREDICHAS



**Cuándo usar:** Para diagnosticar la separabilidad de clases en un modelo de clasificación.



**Propósito:** Visualizar la distribución de probabilidades predichas para cada clase y evaluar la capacidad discriminativa del modelo.


In [ ]:
# DENSIDAD DE PROBABILIDADES PREDICHAS POR CLASE
# Útil para diagnosticar qué tan bien separa el modelo las clases
# Obtener probabilidades de la clase positiva
y_proba = modelo.predict_proba(X_test)[:, 1]

# VISUALIZACIÓN 1: Densidad por clase (usando etiquetas verdaderas)
plt.figure(figsize=(10, 6))
sns.kdeplot(
    y_proba[y_test == 1],   # Probabilidades para clase positiva
    label='Clase 1 (Positiva)',
    fill=True,
    alpha=0.5
)

## 26. VISUALIZACIÓN: CURVAS ROC Y PRECISION-RECALL COMBINADAS



**Cuándo usar:** Para evaluación completa del modelo de clasificación.



**Propósito:** Visualizar ambas curvas en el mismo gráfico para comparar rendimiento desde diferentes perspectivas.


In [ ]:
# GRÁFICO COMBINADO: ROC + PRECISION-RECALL
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# Obtener probabilidades (MÁS APROPIADO que usar predicciones binarias)
y_proba = modelo.predict_proba(X_test)[:, 1]

# Calcular curva ROC
fpr, tpr, roc_thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

# Calcular curva Precision-Recall
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)

# Graficar ambas curvas
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr,
         label=f'ROC curve (AUC = {roc_auc:.3f})',
         linewidth=2, color='blue')
plt.plot(recall, precision,
         label=f'PR curve (AUC = {pr_auc:.3f})',
         linewidth=2, color='orange')

## 27. COMPARACIÓN: PREDICCIONES BINARIAS VS PROBABILIDADES



**Cuándo usar:** Para entender la diferencia entre usar `.predict()` vs `.predict_proba()`.



**Propósito:** Comprender cuándo usar predicciones discretas (0/1) versus probabilidades continuas [0,1].


In [ ]:
# PREDICCIONES BINARIAS VS PROBABILIDADES
# MÉTODO 1: Predicciones binarias (0 o 1)
y_pred_binario = modelo.predict(X_test)
print("Predicciones binarias (primeras 10):")
print(y_pred_binario[:10])
print(f"Tipo: {type(y_pred_binario[0])}")
print(f"Valores únicos: {np.unique(y_pred_binario)}")
# Extraer solo probabilidad de clase positiva (columna 1)
y_proba_positiva = modelo.predict_proba(X_test)[:, 1]
print("Probabilidad de clase positiva (primeras 10):")
print(y_proba_positiva[:10])

## 28. FLUJO COMPLETO: PIPELINE DE CLASIFICACIÓN CON DATOS DESBALANCEADOS



**Cuándo usar:** Como referencia del proceso completo de clasificación.



**Propósito:** Pipeline end-to-end desde datos crudos hasta evaluación del modelo.


In [ ]:
# PIPELINE COMPLETO DE CLASIFICACIÓN
# 1. PREPARACIÓN DE DATOS
columnas_relevantes = ['feature1', 'feature2', 'feature3', 'target']
df_limpio = df[columnas_relevantes].dropna()

# 2. SEPARAR X e y
y = df_limpio['target'].astype(int)
X_categorico = df_limpio.drop(columns=['target'])

# 3. ONE-HOT ENCODING
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=True)
X = encoder.fit_transform(X_categorico)

# 4. TRAIN/TEST SPLIT CON ESTRATIFICACIÓN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)

# 5. ENTRENAR MODELO
from sklearn.linear_model import LogisticRegressionCV
modelo = LogisticRegressionCV(
    Cs=10, cv=5, penalty='l2',
    class_weight='balanced',  # Para datos desbalanceados
    solver='liblinear', random_state=42
).fit(X_train, y_train)

# 6. EVALUACIÓN
from sklearn.metrics import precision_recall_curve, roc_curve, auc

# Obtener probabilidades
y_proba_test = modelo.predict_proba(X_test)[:, 1]

# Calcular métricas
precision, recall, _ = precision_recall_curve(y_test, y_proba_test)
fpr, tpr, _ = roc_curve(y_test, y_proba_test)
pr_auc = auc(recall, precision)
roc_auc = auc(fpr, tpr)

print(f"AUC-PR: {pr_auc:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# IMPORTANTE: También considerar la complejidad/interpretabilidad
# A veces un K menor con score ligeramente inferior es preferible


## 29. SELECCIÓN DE COLUMNAS POR TIPO DE DATO



**Cuándo usar:** Para separar automáticamente variables numéricas, categóricas, booleanas, etc.



**Propósito:** Filtrar columnas según su tipo de dato (int, float, bool, object) para procesamiento diferenciado.


In [ ]:
# SELECCIÓN AUTOMÁTICA POR TIPO DE DATO
# Seleccionar solo columnas numéricas (int y float)
columnas_numericas = df.select_dtypes(include=['int', 'float']).columns
df_numerico = df[columnas_numericas]
print(f"Columnas numéricas ({len(columnas_numericas)}): {columnas_numericas.tolist()}")

# Seleccionar solo columnas booleanas
columnas_booleanas = df.select_dtypes(include=['bool']).columns
df_booleano = df[columnas_booleanas]
print(f"Columnas booleanas ({len(columnas_booleanas)}): {columnas_booleanas.tolist()}")


# CONVERTIR BOOLEANOS A 0/1
df_booleano_int = df_booleano.astype(int)

## 30. INTEGRACIÓN DE DATAFRAMES Y ELIMINACIÓN DE COLUMNAS



**Cuándo usar:** Para combinar diferentes subconjuntos de datos procesados y eliminar columnas irrelevantes.



**Propósito:** Unir horizontalmente dataframes (por columnas) y limpiar variables no deseadas.


In [ ]:
# CONCATENAR DATAFRAMES HORIZONTALMENTE (POR COLUMNAS)
# Ejemplo: Integrar columnas numéricas + booleanas convertidas
df_integrado = pd.concat([df_numerico, df_booleano_int], axis=1)

# axis=0 -> Concatenar verticalmente (añadir filas)
# axis=1 -> Concatenar horizontalmente (añadir columnas)

# ELIMINAR COLUMNAS ESPECÍFICAS
df_limpio = df_integrado.drop(columns=['columna_irrelevante', 'otra_columna'])

## 31. FILTRADO DE COLUMNAS POR PORCENTAJE DE VALORES NULOS



**Cuándo usar:** Para eliminar automáticamente columnas con demasiados valores faltantes.



**Propósito:** Identificar y eliminar columnas según un umbral de porcentaje de NAs (ej: >= 25%).


In [ ]:
# FILTRAR COLUMNAS POR PORCENTAJE DE NAs
# Paso 1: Calcular porcentaje de NAs por columna
porcentaje_nulos = df.isnull().mean() * 100

# Paso 2: Crear máscara booleana (True = columna válida, False = eliminar)
columnas_validas = porcentaje_nulos < 25

# Paso 3: Filtrar dataframe
df_filtrado = df.loc[:, columnas_validas]

print(f"Columnas originales: {df.shape[1]}")
print(f"Columnas eliminadas: {df.shape[1] - df_filtrado.shape[1]}")

## 32. ESTANDARIZACIÓN DE VARIABLE OBJETIVO (Y)



**Cuándo usar:** En regresión cuando se necesita normalizar también la variable objetivo.



**Propósito:** Estandarizar Y (mean=0, std=1) para mejorar convergencia de modelos y comparabilidad de errores.


In [ ]:
# ESTANDARIZACIÓN DE Y (VARIABLE OBJETIVO)
from sklearn.preprocessing import StandardScaler

# MÉTODO 1: Manual (simple)
Y = df['variable_objetivo']
media_Y = Y.mean()
std_Y = Y.std()
Y_escalado = (Y - media_Y) / std_Y

print(f"Media original: {media_Y:.2f}")
print(f"Std original: {std_Y:.2f}")
print(f"Media escalada: {Y_escalado.mean():.6f}")
print(f"Std escalada: {Y_escalado.std():.6f}")

# MÉTODO 2: Con StandardScaler (recomendado para train/test)
# Ventaja: Permite aplicar la misma transformación a test
scaler_Y = StandardScaler()
Y_escalado = scaler_Y.fit_transform(Y.values.reshape(-1, 1)).ravel()

# reshape(-1, 1) convierte array 1D a 2D (requerido por StandardScaler)
# ravel() convierte de vuelta a 1D

# IMPORTANTE: Para train/test, hacer:
# 1. Ajustar scaler solo con Y_train
scaler_Y = StandardScaler()
Y_train_escalado = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1)).ravel()

# 2. Transformar Y_test con el mismo scaler (SIN fit)
Y_test_escalado = scaler_Y.transform(Y_test.values.reshape(-1, 1)).ravel()

## 33. ELASTIC NET PARA REGRESIÓN CON SELECCIÓN DE FEATURES



**Cuándo usar:** Regresión con muchas features, necesidad de selección automática de variables.



**Propósito:** Combinar L1 (Lasso) y L2 (Ridge) para regularización y selección de features simultáneamente.


In [ ]:
# ELASTIC NET CON VALIDACIÓN CRUZADA
# Combina Lasso (L1) y Ridge (L2) regularization
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import explained_variance_score
from scipy.stats import spearmanr

# Preparar datos (después de train/test split y escalado)
# X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled

# Definir grids de hiperparámetros
m = 11
l1_ratios = np.linspace(0, 1, m)  # 0=Ridge puro, 1=Lasso puro
alphas = np.logspace(-10, 10, m)  # Valores de regularización

# Entrenar modelo con CV
modelo = ElasticNetCV(
    alphas=alphas,           # Valores de alpha (fuerza de regularización)
    l1_ratio=l1_ratios,      # Balance entre L1 y L2
    cv=10,                   # 10-fold cross-validation
    random_state=42,         # Reproducibilidad
    max_iter=10000           # Iteraciones máximas (aumentar si no converge)
).fit(X_train_scaled, Y_train_scaled)

# PREDICCIONES Y EVALUACIÓN
# En train
Y_pred_train = modelo.predict(X_train_scaled)
explained_var_train = explained_variance_score(Y_train_scaled, Y_pred_train)
corr_train = spearmanr(Y_train_scaled, Y_pred_train)[0]

# En test
Y_pred_test = modelo.predict(X_test_scaled)
explained_var_test = explained_variance_score(Y_test_scaled, Y_pred_test)
corr_test = spearmanr(Y_test_scaled, Y_pred_test)[0]

# ANÁLISIS DE FEATURES SELECCIONADAS
betas = modelo.coef_
n_features_activas = sum(abs(betas) > 0)

## 34. DIAGNÓSTICO Y VISUALIZACIÓN DE MODELOS DE REGRESIÓN



**Cuándo usar:** Para evaluar visualmente el rendimiento de modelos de regresión.



**Propósito:** Detectar patrones en errores, outliers, y evaluar calidad de predicciones.


In [ ]:
# VISUALIZACIÓN DE DIAGNÓSTICO PARA REGRESIÓN
import matplotlib.pyplot as plt

# GRÁFICO 1: Scatter Plot Real vs Predicho
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(Y_test_scaled, Y_pred_test, alpha=0.5, edgecolors='k', linewidth=0.5)
# Línea de predicción perfecta (45 grados)
plt.plot([Y_test_scaled.min(), Y_test_scaled.max()],
         [Y_test_scaled.min(), Y_test_scaled.max()],
         'r--', linewidth=2, label='Predicción perfecta')
plt.xlabel('Valor Real')
plt.ylabel('Valor Predicho')
plt.title('Real vs Predicho (Test Set)')
plt.legend()
plt.grid(True, alpha=0.3)

# ESTADÍSTICAS DE ERRORES
print("=== ANÁLISIS DE ERRORES ===")
print(f"Error medio: {errores.mean():.4f}")
print(f"Error absoluto medio: {abs(errores).mean():.4f}")
print(f"Error std: {errores.std():.4f}")
print(f"Error máximo: {abs(errores).max():.4f}")

## 35. DETECCIÓN DE OUTLIERS CON PCA



**Cuándo usar:** Para identificar observaciones atípicas que afectan el modelo.



**Propósito:** Usar PCA para visualizar outliers y analizar qué features contribuyen a su anomalía.


In [ ]:
# DETECCIÓN Y ANÁLISIS DE OUTLIERS
from sklearn.decomposition import PCA

# Paso 1: Identificar observación con mayor error
errores = abs(Y_test_scaled - Y_pred_test)
indice_outlier = np.argmax(errores)
print(f"Observación con mayor error: índice {indice_outlier}")
print(f"Error: {errores[indice_outlier]:.4f}")

# Paso 2: Proyectar datos en PCA para visualizar
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_test_scaled)

plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c='lightblue', alpha=0.6, edgecolors='k')
plt.scatter(X_pca[indice_outlier, 0], X_pca[indice_outlier, 1],
            c='red', s=200, marker='X', edgecolors='black', linewidth=2,
            label=f'Outlier (índice {indice_outlier})')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Detección de Outlier con PCA')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Paso 3: Identificar qué feature contribuye más a PC1
# (PC1 suele capturar la mayor variabilidad)
correlaciones = []
for i in range(X_test_scaled.shape[1]):
    corr = np.corrcoef(X_test_scaled[:, i], X_pca[:, 0])[0, 1]
    correlaciones.append(abs(corr))  # Valor absoluto para magnitud

indice_feature_max = np.argmax(correlaciones)
print(f"Feature con mayor correlación con PC1: índice {indice_feature_max}")
print(f"Correlación: {correlaciones[indice_feature_max]:.4f}")s

## 36. K-MEANS CLUSTERING



**Cuándo usar:** Para agrupar datos en clusters sin supervisión.



**Propósito:** Particionar datos en K grupos homogéneos internamente y heterogéneos entre sí.


In [ ]:
# K-MEANS CLUSTERING BÁSICO
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# IMPORTANTE: Estandarizar datos antes de clustering
# K-means es sensible a escalas diferentes entre variables
X = df.drop(columns=['variable_a_excluir'])  # Ej: no usar precio en clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar K-means con K clusters
K = 3  # Número de clusters (luego veremos cómo elegirlo)
kmeans = KMeans(
    n_clusters=K,
    n_init='auto',      # Número de inicializaciones (auto en sklearn reciente)
    random_state=42,    # Reproducibilidad
    max_iter=300        # Iteraciones máximas
)

# Ajustar y obtener etiquetas
labels = kmeans.fit_predict(X_scaled)

# Convertir etiquetas (0, 1, 2...) a (1, 2, 3...)
labels = labels + 1

# Añadir al dataframe original
df['cluster'] = labels

## 37. SELECCIÓN ÓPTIMA DE K CON ÍNDICE DE SILHOUETTE



**Cuándo usar:** Para determinar el número óptimo de clusters en K-means.



**Propósito:** Evaluar la calidad del clustering para diferentes valores de K y seleccionar el mejor.


In [ ]:
# BÚSQUEDA DEL K ÓPTIMO CON SILHOUETTE SCORE
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Rango de K a probar (mínimo 2, máximo a elegir)
K_max = 20
K_range = range(2, K_max + 1)
silhouette_scores = []

# Probar cada K
for k in K_range:
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)
    print(f"K={k}, Silhouette Score={score:.4f}")

# Visualizar resultados
plt.figure(figsize=(10, 5))
plt.plot(K_range, silhouette_scores, marker='o', linestyle='-', linewidth=2)
plt.grid(True, alpha=0.3)
plt.show()

## 38. VISUALIZACIÓN DE CLUSTERS CON PCA



**Cuándo usar:** Para visualizar clusters en 2D cuando los datos tienen muchas dimensiones.



**Propósito:** Proyectar datos clusterizados en 2 componentes principales para evaluación visual.


In [ ]:
# VISUALIZACIÓN DE CLUSTERS CON PCA
from sklearn.decomposition import PCA

# Entrenar solución final con K óptimo
kmeans_final = KMeans(n_clusters=K_optimo, n_init='auto', random_state=42)
labels_final = kmeans_final.fit_predict(X_scaled)

# Ajustar etiquetas para empezar en 1
labels_final = labels_final + 1

# Aplicar PCA para visualización 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Visualizar
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=labels_final,           # Color por cluster
    cmap='tab10',             # Paleta de colores
    alpha=0.6,                # Transparencia
    edgecolors='k',           # Borde negro
    linewidth=0.5,
    s=50                      # Tamaño de puntos
)

## 39. COMPARACIÓN DE MODELOS: CON Y SIN REGULARIZACIÓN



**Cuándo usar:** Para entender el impacto de la regularización.



**Propósito:** Comparar rendimiento de regresión lineal simple vs modelos regularizados.


In [ ]:
# COMPARACIÓN: LINEAR REGRESSION VS ELASTIC NET
from sklearn.linear_model import LinearRegression

# Modelo sin regularización
modelo_simple = LinearRegression()
modelo_simple.fit(X_train_scaled, Y_train_scaled)

# Evaluación
score_train_simple = modelo_simple.score(X_train_scaled, Y_train_scaled)
score_test_simple = modelo_simple.score(X_test_scaled, Y_test_scaled)

print("Regresión Lineal Simple (sin regularización):")
print(f"  R² Train: {score_train_simple:.4f}")
print(f"  R² Test: {score_test_simple:.4f}")
print(f"  Features usadas: {X_train_scaled.shape[1]}/{X_train_scaled.shape[1]}")

print("Elastic Net (con regularización):")
print(f"  R² Train: {explained_var_train:.4f}")
print(f"  R² Test: {explained_var_test:.4f}")
print(f"  Features usadas: {n_features_activas}/{X_train_scaled.shape[1]}")

---

# PARTE IV: MANIPULACIÓN AVANZADA DE DATOS CON PANDAS

---


## 42. EXPLORACIÓN INICIAL DE DATOS



**Cuándo usar:** Siempre al inicio de cualquier análisis.



**Propósito:** Entender la estructura del dataset: dimensiones, tipos de datos y valores faltantes.


In [ ]:
# EXPLORACIÓN INICIAL COMPLETA

# Ver primeras filas
df.head()  # Por defecto muestra 5 filas
df.head(10)  # Mostrar 10 filas

# Ver últimas filas
df.tail()  # Por defecto 5 filas

# Dimensiones del dataset
print(f"Shape: {df.shape}")  # (n_filas, n_columnas)
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

# Tipos de datos de cada columna
print("Tipos de datos:")
print(df.dtypes)

# Información general resumida
df.info()

# Estadísticas descriptivas de columnas numéricas
df.describe()

# Estadísticas de columnas categóricas
df.describe(include='object')

## 43. CREACIÓN DE VARIABLES DERIVADAS CON OPERACIONES



**Cuándo usar:** Para calcular métricas o ratios entre columnas existentes.



**Propósito:** Crear nuevas columnas basadas en operaciones aritméticas entre columnas.


In [ ]:
# CREAR NUEVAS COLUMNAS CON OPERACIONES

# División entre dos columnas
df['ratio'] = df['columna_numerador'] / df['columna_denominador']

# Ejemplo: Comentarios por vista
df['comments_per_view'] = df['comments'] / df['views']

# Operaciones inversas
df['views_per_comment'] = df['views'] / df['comments']

# Multiplicación
df['producto'] = df['col1'] * df['col2']

# Suma
df['total'] = df['col1'] + df['col2'] + df['col3']

# Resta
df['diferencia'] = df['col1'] - df['col2']

# Operaciones más complejas
df['metrica_compuesta'] = (df['col1'] * 0.5 + df['col2'] * 0.3) / df['col3']

## 44. ORDENAMIENTO Y SELECCIÓN DE EXTREMOS



**Cuándo usar:** Para encontrar valores máximos o mínimos de una variable.



**Propósito:** Ordenar el dataframe y extraer las observaciones con valores más altos/bajos.


In [ ]:
# ORDENAMIENTO Y SELECCIÓN DE EXTREMOS

# Ordenar por una columna (ascendente por defecto)
df_ordenado = df.sort_values('columna')

# Ordenar descendente
df_ordenado = df.sort_values('columna', ascending=False)

# Obtener los N valores más ALTOS (tail después de ordenar ascendente)
top_valores = df.sort_values('columna', ascending=True).tail(10)

# Obtener los N valores más BAJOS (head después de ordenar ascendente)
bottom_valores = df.sort_values('columna', ascending=True).head(10)

## 45. VISUALIZACIÓN APROPIADA SEGÚN TIPO DE DATO



**Cuándo usar:** Para elegir el gráfico correcto según el tipo de variable.



**Propósito:** Evitar gráficos inapropiados (ej: time series cuando no hay tiempo).


In [ ]:
# ELEGIR VISUALIZACIÓN APROPIADA

# GRÁFICO DE LÍNEAS: Solo para series temporales o datos ordenados
# ✅ CORRECTO: Para datos sin orden temporal, usar histograma
df['variable_numerica'].plot(kind='hist')

# HISTOGRAMA: Para distribuciones de variables continuas
df['variable_numerica'].plot(kind='hist', bins=30, edgecolor='black')

# BOXPLOT: Para detectar outliers y ver distribución
df['variable'].plot(kind='box')

# BARPLOT: Para conteos de categorías
df['categoria'].value_counts().plot(kind='bar')

# SCATTER: Para relación entre dos variables
df.plot(kind='scatter', x='var1', y='var2')

## 46. FILTRADO CON QUERY Y MÚLTIPLES MÉTODOS



**Cuándo usar:** Para filtrar datos según condiciones.



**Propósito:** Conocer las diferentes formas de filtrar filas en pandas.


In [ ]:
# MÉTODOS DE FILTRADO EN PANDAS

# MÉTODO 1: Indexación booleana (más común)
df_filtrado = df[df['columna'] < 1000]

# MÉTODO 2: .loc con condición
df_filtrado = df.loc[df['columna'] < 1000]

# MÉTODO 3: .query() - Más legible para condiciones complejas
df_filtrado = df.query('columna < 1000')

# Query con múltiples condiciones
df_filtrado = df.query('columna1 < 1000 and columna2 > 50')
df_filtrado = df.query('columna1 < 1000 or columna2 > 50')

# Query con variables externas (usar @)
umbral = 1000
df_filtrado = df.query('columna < @umbral')

# Query con operadores in
categorias = ['cat1', 'cat2', 'cat3']
df_filtrado = df.query('categoria in @categorias')

## 47. CONVERSIÓN Y MANEJO DE FECHAS (DATETIME)



**Cuándo usar:** Cuando se trabaja con fechas en diferentes formatos.



**Propósito:** Convertir timestamps, strings o formatos especiales a datetime de pandas.


In [ ]:
# CONVERSIÓN A DATETIME

# Desde string con formato estándar
df['fecha'] = pd.to_datetime(df['fecha_string'])

# Desde UNIX timestamp (segundos desde 1970-01-01)
df['fecha'] = pd.to_datetime(df['timestamp'], unit='s')

# Verificar tipo de dato
print(df['fecha_procesada'].dtype)

## 48. EXTRACCIÓN DE COMPONENTES DE FECHAS



**Cuándo usar:** Para extraer año, mes, día, etc. de columnas datetime.



**Propósito:** Crear variables derivadas desde fechas para análisis temporal.


In [ ]:
# EXTRACCIÓN DE COMPONENTES CON .dt
# IMPORTANTE: Solo funciona si la columna es tipo datetime

# Extraer año
df['año'] = df['fecha'].dt.year

# Extraer mes (1-12)
df['mes'] = df['fecha'].dt.month

# Extraer día del mes (1-31)
df['dia'] = df['fecha'].dt.day

# Extraer hora, minuto, segundo
df['hora'] = df['fecha'].dt.hour
df['minuto'] = df['fecha'].dt.minute
df['segundo'] = df['fecha'].dt.second

## 49. MUESTREO ALEATORIO CON SAMPLE



**Cuándo usar:** Para inspeccionar datos aleatoriamente o crear subconjuntos.



**Propósito:** Obtener muestras aleatorias para verificación o análisis exploratorio.


In [ ]:
# MUESTREO ALEATORIO

# Obtener N filas aleatorias
muestra = df.sample(10)  # 10 filas aleatorias

# Muestra aleatoria de columnas específicas
muestra = df[['col1', 'col2', 'col3']].sample(5)

# Muestra con reemplazo (puede repetir filas)
muestra = df.sample(100, replace=True)

## 50. BÚSQUEDA DE PATRONES EN STRINGS CON .str.contains()



**Cuándo usar:** Para filtrar o verificar presencia de texto en columnas de strings.



**Propósito:** Detectar si un patrón o palabra está presente en valores de texto.


In [ ]:
# BÚSQUEDA DE PATRONES EN STRINGS

# Verificar si una palabra está en cada valor
contiene_palabra = df['columna_texto'].str.contains('palabra')
# Devuelve Serie booleana (True/False)

# Filtrar filas que contienen la palabra
df_filtrado = df[df['columna_texto'].str.contains('palabra')]

# Case-insensitive (ignorar mayúsculas/minúsculas)
df['columna'].str.contains('palabra', case=False)

# Búsqueda con expresiones regulares
df['columna'].str.contains('patron.*regex', regex=True)

# EJEMPLO PRÁCTICO: Verificar completitud de datos
# ¿Todas las filas tienen cierta palabra en sus ratings?
tiene_rating_funny = df['ratings'].str.contains('Funny').all()

if tiene_rating_funny:
    print("✓ Todas las filas tienen rating 'Funny'")
else:
    print("✗ Algunas filas no tienen rating 'Funny'")
    n_sin_funny = (~df['ratings'].str.contains('Funny')).sum()
    print(f"Filas sin 'Funny': {n_sin_funny}")

## 51. CONVERSIÓN DE STRINGS A ESTRUCTURAS DE DATOS CON ast.literal_eval



**Cuándo usar:** Cuando se tienen strings que representan listas, diccionarios o estructuras de Python.



**Propósito:** Convertir strings con formato de Python ("[1, 2, 3]", "{' key': 'value'}") a objetos reales.


In [ ]:
# CONVERSIÓN DE STRINGS A ESTRUCTURAS DE DATOS

import ast
# Ver tipo original
print(type(df['columna_con_lista'][0]))  # <class 'str'>

# Ejemplo simple
string_lista = "[1, 2, 3, 4, 5]"
lista_real = ast.literal_eval(string_lista)
print(type(lista_real))  # <class 'list'>
print(lista_real)  # [1, 2, 3, 4, 5]

# APLICAR A TODA UNA COLUMNA con .apply()
df['columna_convertida'] = df['columna_string'].apply(ast.literal_eval)

# VERIFICAR CONVERSIÓN
print("Antes:")
print(type(df['columna_string'][0]))  # str
print("Después:")
print(type(df['columna_convertida'][0]))  # list o dict

# PARTE III: PREPROCESAMIENTO AVANZADO Y CLUSTERING

## 53. ANÁLISIS DE FRECUENCIAS Y ORDENAMIENTO CON VALUE_COUNTS



**Cuándo usar:** Para contar ocurrencias de valores únicos en una columna.



**Propósito:** Obtener distribuciones de frecuencias y ordenarlas apropiadamente.


In [ ]:
# VALUE_COUNTS Y ORDENAMIENTO

# Conteo básico de frecuencias
conteos = df['columna_categorica'].value_counts()
print(conteos)

# Con proporciones (porcentajes)
proporciones = df['columna_categorica'].value_counts(normalize=True)
porcentajes = df['columna_categorica'].value_counts(normalize=True) * 100

# VISUALIZAR VALUE_COUNTS
# Gráfico de barras por frecuencia
df['categoria'].value_counts().plot(kind='bar')
plt.title('Frecuencia por categoría')
plt.show()

# Gráfico de líneas temporal (después de sort_index)
df['año'].value_counts().sort_index().plot()
plt.title('Evolución temporal')
plt.xlabel('Año')
plt.ylabel('Cantidad')
plt.show()

# IMPORTANTE: Diferencia entre sort_values y sort_index
# sort_values() → ordena por los valores (frecuencias)
# sort_index() → ordena por el índice (las categorías/años)

## 54. AGREGACIONES MÚLTIPLES CON AGG



**Cuándo usar:** Para calcular múltiples estadísticas simultáneamente sobre grupos.



**Propósito:** Obtener varias métricas (count, mean, sum, etc.) en una sola operación.


In [ ]:
# AGREGACIONES MÚLTIPLES CON .agg()

# CASO 1: Varias funciones sobre UNA columna
resultado = df.groupby('categoria')['valor'].agg(['count', 'mean', 'sum'])
print(resultado)

# Funciones disponibles como strings:
# 'count', 'sum', 'mean', 'median', 'std', 'var', 'min', 'max', 'first', 'last'

# CASO 2: Diferentes funciones sobre DIFERENTES columnas
resultado = df.groupby('categoria').agg({
    'columna1': 'mean',
    'columna2': 'sum',
    'columna3': ['min', 'max'],
    'columna4': 'count'
})

# ORDENAR RESULTADOS
resultado = (df.groupby('evento')['visualizaciones']
             .agg(['count', 'mean', 'sum'])
             .sort_values('mean', ascending=False))

# RESETEAR ÍNDICE para mejor manipulación
resultado = (df.groupby('evento')['visualizaciones']
             .agg(['count', 'mean', 'sum'])
             .reset_index()
             .sort_values('mean', ascending=False))

## 55. FILTRADO CON .isin() PARA MÚLTIPLES VALORES



**Cuándo usar:** Para filtrar filas donde una columna contiene uno de varios valores posibles.



**Propósito:** Seleccionar subconjuntos basados en listas de valores permitidos.


In [ ]:
# FILTRADO CON .isin()

# Definir lista de valores permitidos
valores_deseados = ['valor1', 'valor2', 'valor3']

# Filtrar filas que tienen alguno de esos valores
df_filtrado = df[df['columna'].isin(valores_deseados)]

## 56. ANÁLISIS DE COLUMNAS CATEGÓRICAS CON describe()



**Cuándo usar:** Para entender distribuciones de variables categóricas.



**Propósito:** Obtener estadísticas sobre textos: valores únicos, más frecuente, etc.


In [ ]:
# ESTADÍSTICAS DE VARIABLES CATEGÓRICAS

# Para variables numéricas (default)
df.describe()

# Para variables categóricas (tipo object/string)
df.describe(include='object')

# Describe de una columna específica
df['columna_categorica'].describe()

# Información que proporciona describe() para categóricas:
# - count: Número de valores no-NaN
# - unique: Número de valores únicos
# - top: Valor más frecuente
# - freq: Frecuencia del valor más frecuente

# EJEMPLO
stats = df['ocupacion'].describe()
print(f"Total registros: {stats['count']}")
print(f"Ocupaciones únicas: {stats['unique']}")
print(f"Ocupación más común: {stats['top']}")
print(f"Aparece {stats['freq']} veces")

# MÉTODOS COMPLEMENTARIOS
# Número de valores únicos
n_unicos = df['columna'].nunique()

# Lista de valores únicos
valores = df['columna'].unique()

# Valor más frecuente
mas_frecuente = df['columna'].mode()[0]